In [2]:
#Import packages

import numpy as np
import pandas as pd
from PIL import Image
import os
import path

import torch
import torchvision
from torch.utils import data
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [3]:
def target_to_oh(target):
    NUM_CLASS = 5  # hard code here, can do partial
    one_hot = torch.eye(NUM_CLASS)[target]
    return one_hot

In [4]:
transform_bunch = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.ToTensor()])
#                                       transforms.Normalize((0.5139, 0.2727, 0.0891), (0.1533, 0.0909, 0.0400))])

In [7]:
ds = torchvision.datasets.ImageFolder("../archive/gaussian_filtered_images/gaussian_filtered_images", transform_bunch, target_transform = target_to_oh)

In [8]:
for x,y in ds:
    print(y)
    break

tensor([1., 0., 0., 0., 0.])


In [9]:
percent = 0.2

split = int(len(ds) * percent)

indices = list(range(len(ds)))

In [10]:
split

732

In [11]:
train_sampler = SubsetRandomSampler(indices[split:])

valid_sampler = SubsetRandomSampler(indices[:split])

In [12]:
bs = 100

dl = data.DataLoader(ds, bs, sampler=train_sampler)

valid_dl = data.DataLoader(ds, bs, sampler=valid_sampler)

In [13]:
xx = 0
yy = 0

for x,y in valid_dl:
    print(x.shape,y.shape)
    xx = x
    yy = y
    break

torch.Size([100, 3, 224, 224]) torch.Size([100, 5])


In [14]:
yy

tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0

In [15]:
#looks like something is off... Yup --> We can just transform it at the end
ds.class_to_idx

{'Mild': 0, 'Moderate': 1, 'No_DR': 2, 'Proliferate_DR': 3, 'Severe': 4}

In [16]:
#Now I'll be making the model! It'll take elements off of nn.Module
class cnnmaker(nn.Module):
    #Initialization will only take in input channels
    def __init__(self, input_channels):
        super().__init__()
        
        #I looked at what the resnet architechture looked like and just implmented the block types (not the residual part just yet)
        def block(in_chan):
            return nn.Sequential(nn.Conv2d(in_chan, in_chan * 2, 3, 2, 1, padding_mode = "reflect"),
                                nn.BatchNorm2d(in_chan*2),
                                nn.ReLU())
        
        self.model = nn.Sequential(block(input_channels),
                                  block(input_channels*2),
                                  block(input_channels*4),
                                  block(input_channels*8),
                                  block(input_channels*16))
        
        self.second_model = nn.Sequential(nn.Linear(4704, 500),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(500, 100),
                                         nn.Dropout(),
                                         nn.ReLU(),
                                         nn.Linear(100, 5))
    #The forward pass when you call it   
    def forward(self, images):
        pre_proc = self.model(images)
#         print(pre_proc.shape)
        
        formatted = torch.reshape(pre_proc, (images.shape[0], -1))
        
        return self.second_model(formatted)

In [17]:
model = cnnmaker(3)

In [18]:
model

cnnmaker(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(12, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): Sequential(
      (0):

In [19]:
#Look at how many parameters
total = 0
for param in model.parameters():
    if param.requires_grad:
        total += param.numel()

print(total)

2458905


In [20]:
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [22]:
model = model.to('cpu')
loss_func = loss_func.to('cpu')

In [23]:
def accuracy(preds, target):
    correct = (preds == target).float()
    accuracy = correct.sum() / len(correct)
    return accuracy

In [30]:
def train():
    model.train()
    
    for image, label in dl:
        image, label = image.type(dtype=torch.FloatTensor), label.type(dtype=torch.FloatTensor)
        optimizer.zero_grad()
        prediction = model(image)
        
        loss = loss_func(prediction, label)
        
        loss.backward()
        
        optimizer.step()
    
    with torch.no_grad():
        total_iter = 0
        total_acc = 0
        for image, label in valid_dl: 
            image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
            prediction = model(image)
            total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
            total_iter += 1
            if total_iter == 7:
                print("valid",total_acc/total_iter)
                total_iter = 0
                total_acc = 0
                for image, label in dl: 
                    image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
                    prediction = model(image)
                    total_acc += accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
                    total_iter += 1
                    if total_iter == 7:
                        print(total_acc/total_iter)
                        return "done"

In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)

In [32]:
for _ in range(4):
    train()

valid tensor(0.1914)
tensor(0.7314)
valid tensor(0.2957)
tensor(0.7886)
valid tensor(0.3000)
tensor(0.7729)
valid tensor(0.2814)
tensor(0.7743)


In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [34]:
for _ in range(10):
    train()

valid tensor(0.3714)
tensor(0.8057)
valid tensor(0.2871)
tensor(0.8114)
valid tensor(0.3214)
tensor(0.7957)
valid tensor(0.3400)
tensor(0.7957)
valid tensor(0.3400)
tensor(0.7886)
valid tensor(0.3343)
tensor(0.7943)
valid tensor(0.4057)
tensor(0.8014)
valid tensor(0.3529)
tensor(0.7871)
valid tensor(0.3329)
tensor(0.8014)
valid tensor(0.3657)
tensor(0.8100)


In [35]:
for _ in range(20):
    train()

valid tensor(0.3171)
tensor(0.8043)
valid tensor(0.3629)
tensor(0.8286)
valid tensor(0.3771)
tensor(0.8243)
valid tensor(0.3929)
tensor(0.8071)
valid tensor(0.3843)
tensor(0.7771)
valid tensor(0.3743)
tensor(0.8200)
valid tensor(0.3886)
tensor(0.8071)
valid tensor(0.3829)
tensor(0.7900)
valid tensor(0.3729)
tensor(0.8086)
valid tensor(0.3929)
tensor(0.8029)
valid tensor(0.4014)
tensor(0.8329)
valid tensor(0.3300)
tensor(0.8429)
valid tensor(0.3600)
tensor(0.8229)
valid tensor(0.3471)
tensor(0.8343)
valid tensor(0.3629)
tensor(0.8486)
valid tensor(0.3629)
tensor(0.8386)
valid tensor(0.3386)
tensor(0.8271)
valid tensor(0.3757)
tensor(0.8271)
valid tensor(0.3571)
tensor(0.8400)
valid tensor(0.3500)
tensor(0.8143)


In [36]:
for _ in range(20):
    train()

valid tensor(0.3171)
tensor(0.8457)
valid tensor(0.3771)
tensor(0.8214)
valid tensor(0.3457)
tensor(0.8643)
valid tensor(0.2986)
tensor(0.8586)
valid tensor(0.3143)
tensor(0.8543)
valid tensor(0.3571)
tensor(0.8814)
valid tensor(0.2943)
tensor(0.8757)
valid tensor(0.3443)
tensor(0.8657)
valid tensor(0.3114)
tensor(0.8700)
valid tensor(0.3100)
tensor(0.8843)
valid tensor(0.3071)
tensor(0.8957)
valid tensor(0.3171)
tensor(0.8900)
valid tensor(0.2714)
tensor(0.9100)
valid tensor(0.2857)
tensor(0.9029)
valid tensor(0.2929)
tensor(0.9129)
valid tensor(0.2500)
tensor(0.8971)
valid tensor(0.2714)
tensor(0.8957)
valid tensor(0.2943)
tensor(0.9129)
valid tensor(0.2557)
tensor(0.9200)
valid tensor(0.3000)
tensor(0.9200)


In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [38]:
for _ in range(20):
    train()

valid tensor(0.2957)
tensor(0.9471)
valid tensor(0.2843)
tensor(0.9500)
valid tensor(0.2971)
tensor(0.9471)
valid tensor(0.3029)
tensor(0.9314)
valid tensor(0.2686)
tensor(0.9529)
valid tensor(0.2671)
tensor(0.9371)
valid tensor(0.2729)
tensor(0.9414)
valid tensor(0.2700)
tensor(0.9400)
valid tensor(0.2743)
tensor(0.9543)
valid tensor(0.2771)
tensor(0.9357)
valid tensor(0.2943)
tensor(0.9714)
valid tensor(0.2743)
tensor(0.9471)
valid tensor(0.2643)
tensor(0.9643)
valid tensor(0.2800)
tensor(0.9614)
valid tensor(0.2771)
tensor(0.9557)
valid tensor(0.2786)
tensor(0.9600)
valid tensor(0.2786)
tensor(0.9514)
valid tensor(0.2757)
tensor(0.9714)
valid tensor(0.2614)
tensor(0.9543)
valid tensor(0.2771)
tensor(0.9671)


In [39]:
xxx = 0
yyy = 0

for x,y in dl:
    print(x.shape,y.shape)
    xxx = x
    yyy = y
    break

torch.Size([100, 3, 224, 224]) torch.Size([100, 5])


In [40]:
yyy = yyy.type(dtype=torch.FloatTensor)
torch.argmax(yyy, dim = 1).float()

tensor([2., 2., 3., 2., 1., 1., 2., 2., 1., 2., 2., 2., 2., 3., 2., 4., 2., 2.,
        2., 2., 2., 1., 2., 3., 2., 1., 2., 1., 2., 4., 1., 2., 2., 1., 2., 2.,
        2., 2., 4., 4., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 2.,
        2., 2., 2., 4., 4., 2., 2., 2., 3., 2., 2., 1., 2., 2., 2., 1., 4., 4.,
        1., 1., 1., 1., 3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 1., 2., 4., 2.,
        2., 3., 2., 2., 2., 2., 3., 2., 1., 1.])

In [41]:
xxx = xxx.type(dtype=torch.FloatTensor)
prediction = model(xxx)
torch.argmax(prediction, dim = 1).float()


tensor([2., 2., 3., 2., 1., 3., 2., 2., 1., 2., 2., 2., 2., 3., 2., 4., 2., 2.,
        2., 2., 2., 1., 2., 3., 2., 1., 2., 1., 2., 4., 1., 2., 2., 1., 2., 2.,
        2., 2., 4., 3., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 2.,
        2., 2., 2., 4., 1., 2., 2., 2., 3., 2., 2., 1., 2., 2., 2., 1., 4., 4.,
        1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 4., 2., 2., 1., 2., 4., 2.,
        2., 3., 2., 2., 2., 2., 1., 2., 1., 1.])

In [42]:
accuracy(torch.argmax(prediction, dim = 1).float(),torch.argmax(yyy, dim = 1).float())

tensor(0.9400)

In [43]:
with torch.no_grad():
    total_iter = 0
    total_acc = 0
    for image, label in dl: 
        image, label = image.to('cpu', dtype=torch.float), label.to('cpu', dtype=torch.float)
        prediction = model(image)
        h = accuracy(torch.argmax(prediction, dim = 1).float(), torch.argmax(label, dim = 1).float())
        print(h)
        total_acc += h
        total_iter += 1
        if total_iter == 20:
            print(total_acc/total_iter)

tensor(0.9800)
tensor(0.9800)
tensor(0.9600)
tensor(0.9900)
tensor(0.9800)
tensor(0.9900)
tensor(0.9400)
tensor(0.9500)
tensor(0.9400)
tensor(0.9500)
tensor(0.9700)
tensor(0.9200)
tensor(0.9700)
tensor(0.9600)
tensor(0.9600)
tensor(0.9700)
tensor(0.9100)
tensor(0.9800)
tensor(0.9600)
tensor(0.9700)
tensor(0.9615)
tensor(0.9600)
tensor(0.9700)
tensor(0.9700)
tensor(0.9300)
tensor(0.9500)
tensor(0.9400)
tensor(0.9700)
tensor(0.9200)
tensor(0.9700)
tensor(1.)
